# <u>Face Recognition System

Nowadays there are many ways of authenticating yourself, like using password, retina scan, fingerprint etc. Face can also be used for this purpose. In this notebook we will make a face recognition system using Siamese network.
This is different from face verification where the task is to know whether given two input images are same or not.
Here the task is see whether the given input image is of any person who is registered with the system or not. There can be multiple users registered with the system.

The advantage of **Siamese Network** is that it allows a way to do this sort of verification task with very little user data, as it is quite unreasonable to train using thousands of images for each user. Here we will be using **FaceNet Model**.

FaceNet learns a neural network that encodes a face image into a vector of 128 numbers. So by comparing two such vectors, we can then determine if two pictures are of the same person.

In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first')

import pickle
import cv2
import os.path
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from utility import *
from inception_blocks_v2 import *
from webcam_utility import *
np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


## Model
The model makes an encoding vector consisting of 128 numbers for the input image. Two encodings are compared and if the two encodings are similar then we say that the two images are of the same person otherwise they are different. 
The model uses **Triplet loss function**. The aim is to minimize this function.

In [2]:
# triplet loss function
#  y_pred - list containing three objects:
#         anchor(None, 128) -- encodings for the anchor images
#         positive(None, 128) -- encodings for the positive images
#         negative(None, 128) -- encodings for the negative images
def triplet_loss(y_true, y_pred, alpha = 0.2):
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
    # triplet formula components
    pos_dist = tf.reduce_sum( tf.square(tf.subtract(y_pred[0], y_pred[1])) )
    neg_dist = tf.reduce_sum( tf.square(tf.subtract(y_pred[0], y_pred[2])) )
    basic_loss = pos_dist - neg_dist + alpha
    
    loss = tf.maximum(basic_loss, 0.0)
   
    return loss

### Loading the Model
The model outputs a vector of 128 numbers which represent encoding for the given input image. We will be using this encoding vector for comparing two images.
#### Input
- This network takes as input 96x96 RGB image as its input. Specifically, inputs a tensor of shape $(m, n_C, n_H, n_W)$ , where $n_C$ = channel.

#### Output
- A matrix of shape **(m, 128)** where the 128 numbers are the encoding values for $ith$ image.

In [3]:
# load the model
FRmodel = faceRecoModel(input_shape=(3, 96, 96))
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])

### Loading the trained weights
We will be using a pretrained model since it requires a lot of time and data for training such a model.

In [4]:
# load saved weights
load_weights_from_FaceNet(FRmodel)

In [6]:
FRmodel.save('models/model.h5')

In [27]:
# load model
#FRmodel = load_model('models/model.h5')
FRmodel.load_weights('models/weights.h5')

In [3]:
from keras.models import load_model
FRmodel = load_model('models/model.h5', custom_objects={'triplet_loss': triplet_loss})

In [5]:
# save model weights
FRmodel.save_weights(r'models/weights.h5')

In [26]:
FRmodel.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 3, 96, 96)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 3, 102, 102)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 64, 48, 48)   9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn1 (BatchNormalization)        (None, 64, 48, 48)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

### User Database

We will create a database of registered. For this we will use a simple dictionary and map each registered user with his/her face encoding.

In [4]:
# initialize the user database
def ini_user_database():
    # check for existing database
    if os.path.exists('database/user_dict.pickle'):
        with open('database/user_dict.pickle', 'rb') as handle:
            user_db = pickle.load(handle)   
    else:
        # make a new one
        # we use a dict for keeping track of mapping of each person with his/her face encoding
        user_db = {}
     
    return user_db

In [5]:
# adds a new user face to the database using his/her image stored on disk using the image path
def add_user_img_path(user_db, FRmodel, name, img_path):
    if name not in user_db: 
        user_db[name] = img_to_encoding(img_path, FRmodel)
        # save the database
        with open('database/user_dict.pickle', 'wb') as handle:
                pickle.dump(user_db, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print('User ' + name + ' added successfully')
    else:
        print('The name is already registered! Try a different name.........')

In [6]:
# adds a new user using image taken from webcam
def add_user_webcam(user_db, FRmodel, name):
    # we can use the webcam to capture the user image then get it recognized
    face_found = detect_face(user_db, FRmodel)

    if face_found:
        resize_img("saved_image/1.jpg")
        if name not in user_db: 
            add_user_img_path(user_db, FRmodel, name, "saved_image/1.jpg")
        else:
            print('The name is already registered! Try a different name.........')
    else:
        print('There was no face found in the visible frame. Try again...........')

In [7]:
# deletes a registered user from database
def delete_user(user_db, name):
    popped = user_db.pop(name, None)
    
    if popped is not None:
        print('User ' + name + ' deleted successfully')
        # save the database
        with open('database/user_dict.pickle', 'wb') as handle:
                pickle.dump(user_db, handle, protocol=pickle.HIGHEST_PROTOCOL)
    elif popped == None:
        print('No such user !!')

### Add or delete user 

In [8]:
# we use a dict for keeping track of ampping of each person with his/her face encoding
user_db = ini_user_database()

In [34]:
# add a user
add_user_img_path(user_db, FRmodel, "susanta", "images/10.jpg")

User susanta added successfully


In [115]:
add_user_webcam(user_db, FRmodel, "susanta")

User susanta added successfully


In [33]:
delete_user(user_db, "susanta")

User susanta deleted successfully


### Putting everything together
For making this face recognition system we are going to take the input image, find its encoding and then see if there is any similar encoding in the database or not. We define a threshold value to decide whether the two images are similar or not based on the similarity of their encodings.

In [9]:
# recognize the input user face encoding by checking for it in the database
def find_face(image_path, database, model, threshold = 0.6):
    # find the face encodings for the input image
    encoding = img_to_encoding(image_path, model)
    
    min_dist = 99999
    # loop over all the recorded encodings in database 
    for name in database:
        # find the similarity between the input encodings and claimed person's encodings using L2 norm
        dist = np.linalg.norm(np.subtract(database[name], encoding) )
        # check if minimum distance or not
        if dist < min_dist:
            min_dist = dist
            identity = name

    if min_dist > threshold:
        print("User not in the database.")
        identity = 'Unknown Person'
    else:
        print ("Hi! " + str(identity) + ", L2 distance: " + str(min_dist))
        
    return min_dist, identity

In [10]:
# takes an input image and performs face recognition on it
def do_face_recognition(user_db, FRmodel, threshold = 0.7, save_loc = "saved_image/1.jpg"):
    # we can use the webcam to capture the user image then get it recognized
    face_found = detect_face(user_db, FRmodel)

    if face_found:
        resize_img("saved_image/1.jpg")
        min_dist, identity = find_face("saved_image/1.jpg", user_db, FRmodel, threshold)
        print(min_dist)
        print(identity)
    else:
        print('There was no face found in the visible frame. Try again...........')
        

### <u>Run the face recognition system

In [14]:
do_face_recognition(user_db, FRmodel, 0.7, "saved_image/1.jpg")

User not in the database.
0.998339
Unknown Person


In [114]:
detect_face_realtime(user_db, FRmodel, threshold = 0.7)

**************** Enter "q" to quit **********************
Unknown user detected !
Unknown user detected !
Unknown user detected !


In [11]:
def find_face1(encoding, database, model, threshold=0.6):
    min_dist = 99999
    # loop over all the recorded encodings in database
    for name in database:
        # find the similarity between the input encodings and claimed person's encodings using L2 norm
        dist = np.linalg.norm(np.subtract(database[name], encoding))
        # check if minimum distance or not
        if dist < min_dist:
            min_dist = dist
            identity = name

    if min_dist > threshold:
        #print("User not in the database.")
        identity = 'Unknown Person'
    else:
        #print("Hi! " + str(identity) + ", L2 distance: " + str(min_dist))
        pass

    return min_dist, identity

In [ ]:
path = 'your path here'
resize_img(path)
encoding = img_to_encoding(path, FRmodel)

threshold = 0.7
min_dist = 99999
# loop over all the recorded encodings in database
for name in database:
    # find the similarity between the input encodings and claimed person's encodings using L2 norm
    dist = np.linalg.norm(np.subtract(database[name], encoding))
    # check if minimum distance or not
    if dist < min_dist:
        min_dist = dist
        identity = name

if min_dist > threshold:
    print("User not in the database.")
    identity = 'Unknown Person'
else:
    print("Hi! " + str(identity) + ", L2 distance: " + str(min_dist))
    
print('Identity: ' + str(identity))
print('Min distance: ' + str(min_dist))

In [15]:
path = 'saved_image/new.jpg'
resize_img(path)
encoding = img_to_encoding(path, FRmodel)
min_dist, identity = find_face1(encoding, user_db, FRmodel, threshold=0.6)
print(min_dist)

1.15049


In [13]:
for i in range(11):
    path = 'saved_image/' + str(i+1) + '.jpg'
    resize_img(path)
    encoding = img_to_encoding(path, FRmodel)
    min_dist, identity = find_face1(encoding, user_db, FRmodel, threshold=0.6)
    print(str(i+1) + ': ' + identity + ' ===> ' +str(min_dist))

1: Unknown Person ===> 0.998339
2: Unknown Person ===> 1.00343
3: Unknown Person ===> 0.771419
4: Unknown Person ===> 1.11724
5: susanta ===> 0.440618
6: Unknown Person ===> 0.623287
7: susanta ===> 0.368696
8: Unknown Person ===> 1.08845
9: susanta ===> 0.313724
10: susanta ===> 0.00598125
11: Unknown Person ===> 0.989836


### References:
- A lot of code is based on the assignment from Convolutional Neural Networks Specialization by Deeplearning.ai on Coursera.<br>
https://www.coursera.org/learn/convolutional-neural-networks/home/welcome 
- Florian Schroff, Dmitry Kalenichenko, James Philbin (2015). [FaceNet: A Unified Embedding for Face Recognition and Clustering](https://arxiv.org/pdf/1503.03832.pdf)
- Yaniv Taigman, Ming Yang, Marc'Aurelio Ranzato, Lior Wolf (2014). [DeepFace: Closing the gap to human-level performance in face verification](https://research.fb.com/wp-content/uploads/2016/11/deepface-closing-the-gap-to-human-level-performance-in-face-verification.pdf) 
- The pretrained model used is inspired by Victor Sy Wang's implementation and was loaded using his code: https://github.com/iwantooxxoox/Keras-OpenFace.
- A lot of inspiration from the official FaceNet github repository: https://github.com/davidsandberg/facenet 